# Particle Set


Simulate trajectories of a particle cloud in a two-dimensional flow field.
A doubly-periodic domain and randomly-generated flow fields are initially used.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

Exercises:
- change the initial distribution of partices
- increase the duration of the trajectories simulation
- treat the non-periodic domain case by padding `u,v` with zeros
- replace `u,v` with your own two-dimensional flow fields

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1. Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))

Main.##256.isosurface

## 2. Flow Field

The `u,v` arrays below can be replaced with any other pair provided by the user.
A couple of important considerations, however:

- `u,v` are staggered on a C-grid; by `-0.5` grid point in direction `1` for `u` (`2` for `v`)
 from the grid cell center (0.5,0.5)
- `u,v` here derive from streamfunction `ϕ`, defined at the corner point, which ensures that
 the resulting `u,v` is non-divergent, purely rotational, over the C-grid domain.
In brief:

```
u=-(circshift(ϕ, (0,-1))-ϕ)
v=(circshift(ϕ, (-1,0))-ϕ)
```

In [2]:
u,v,ϕ=setup_random_flow()

([0.007754609270090321 0.011138762776975716 … -0.0016620223490348696 0.005027718814079402; 0.010923738446552617 0.01154688774194966 … 0.0013635686525768353 0.0055273269353839916; … ; 0.00488202341041126 0.008968873557041679 … -0.005494208492392447 0.0018977679457618801; 0.007391645493832963 0.008167861953082042 … -0.0028106076835923005 0.00198006434659731], [0.004275654805263748 0.0011065256288014524 … 0.007800853928180043 0.004775262926568338; 0.0016867999307303705 0.0020326668351220017 … 0.004035096493176432 0.00457471139005905; … ; 0.0036770005658316035 0.0011673784824099004 … 0.00644289777546718 0.0037592969666670334; 0.0030565430180745923 0.0026935792418172344 … 0.007252782820114115 0.006104197485556684], [0.10114683597771659 0.09339222670762627 … 0.10451253244276112 0.106174554791796; 0.10542249078298034 0.09449875233642772 … 0.11231338637094117 0.11094981771836433; … ; 0.0944132923938104 0.08953126898339914 … 0.09081685184717983 0.09631106033957228; 0.098090292959642 0.090698647

If user were to start with collocated velocity (`uC,vC` at the grid cell center) then
one can easily obtain the staggered velocity (`u,v`) as follows. These may contain both
[rotational and divergent](https://en.wikipedia.org/wiki/Helmholtz_decomposition) components.

```
u=0.5*(circshift(uC, (0,1))+uC)
v=0.5*(circshift(vC, (1,0))+vC)
```

## 3. Initialize Individuals

For example, we can initialize 100 particles within a central subdomain as follows.

In [3]:
np,nq=size(u)
x=np*(0. .+ 1.0*rand(1000))
y=nq*(0. .+ 1.0*rand(1000));

The `setup_point_cloud` function then wraps everything in the `Individuals` data structure.

In [4]:
𝐼=setup_point_cloud(u,v,X=x,Y=y)
𝐼.𝑃.𝑇[2]=10.
𝐼.🔴

,ID,x,y,t
,Int64,Float64,Float64,Float64


## 3. Compute Trajectories

The time period is `𝐼.𝑃.𝑇` by default, unless `∫!(𝐼,𝑇)` is called instead.

In [5]:
∫!(𝐼)

1×1000 Array{Array{Float64,1},2}:
 [1.86867, 17.298, 1.0]  [3.14696, 2.70572, 1.0]  …  [10.5006, 13.6474, 1.0]

## 4. Plot Results

For example, generate a simple animation:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*